In [1]:
experiment_title = "ALITHEA_Full_service_Multiomics_Botrytis_cinerea"
s3_folder_name = "20250116_PicassoXBridge_LYON_lysate_fullservice"
start_genestack_table = "ALITHEA_Full_service_Multiomics_Botrytis_cinerea.xlsx"

In [2]:
import os
import glob
base_dir = "/home/ubuntu/batch2_alithea"
sit_table_alithea = f"{base_dir}/s3_mount/Experiments/{s3_folder_name}/AMP0313/sample_info_minimal_SIT.xlsx"
os.system(f". {base_dir}/s3fs.sh")
meta_folder = f"{base_dir}/s3_mount/Experiments/{s3_folder_name}/MetaData"
raw_counts = f"{meta_folder}/full_raw_matrix_and_meta.csv"
norm_counts = f"{meta_folder}/full_norm_matrix_and_meta.csv"
metadata = list(glob.glob(f"{meta_folder}/*.xlsx"))[0]

mkdir: cannot create directory ‘s3_mount’: File exists


In [4]:
import pandas as pd 

df = pd.read_csv(norm_counts, index_col=0)
df["Metadata_sample_id"]= df["Metadata_plate"] + "_" + df["Metadata_Wells"]+ "_" + df["Metadata_treatments"]+ "_" + df["Metadata_concentration"].astype(str)
meta_cols = [ col for col in df.columns if col.startswith("Metadata_")]
gene_cols = [ col for col in df.columns if not col.startswith("Metadata_")]
df.index = df["Metadata_sample_id"]
df[gene_cols].T.to_csv(f"{experiment_title}.counts.deseq.normalised.tsv", index=True, sep = "\t")
print(df.head())


                                                   Bcin01g00010  Bcin01g00020  \
Metadata_sample_id                                                              
L111224TS01_01_A01_DMSO_1.0                           38.886375      8.045457   
L111224TS01_01_A02_BCS-AK75639_3.703703703703704       5.367342      2.236393   
L111224TS01_01_A03_BCS-CZ69404_0.1371742112482853      5.229848      2.816072   
L111224TS01_01_A04_BCS-DG83654_0.0152415790275872      5.885352      2.140128   
L111224TS01_01_A05_BCS-AA31476_11.111111111111112      3.669512      1.467805   

                                                   Bcin01g00030  Bcin01g00040  \
Metadata_sample_id                                                              
L111224TS01_01_A01_DMSO_1.0                           69.056838     22.460234   
L111224TS01_01_A02_BCS-AK75639_3.703703703703704       8.721931      2.460032   
L111224TS01_01_A03_BCS-CZ69404_0.1371742112482853     12.068880      2.816072   
L111224TS01_01_A04_BCS-DG83

In [10]:

df = pd.read_csv(raw_counts, index_col=0)
gene_cols = [ col for col in df.columns if not col.startswith("Metadata_")]
df["Metadata_sample_id"]= df["Metadata_plate"] + "_" + df["Metadata_Wells"]+ "_" + df["Metadata_treatments"]+ "_" + df["Metadata_concentration"].astype(str)
meta_cols = [ col for col in df.columns if col.startswith("Metadata_")]
df.index = df["Metadata_sample_id"]
df[gene_cols].T.to_csv(f"{experiment_title}.counts.raw.tsv", index=True, sep = "\t")
print(df.head())

df = df[meta_cols]

                                 AOB78_gp01  AOB78_gp02  AOB78_gp03  \
Metadata_sample_id                                                    
plate1_24h_A01_DMSO_25.0                  5         117          27   
plate1_24h_A02_DMSO_25.0                  2         131          20   
plate1_24h_A03_BCS-DH19921_25.0           2          74          15   
plate1_24h_A04_BCS-DH19921_2.5            6         136          28   
plate1_24h_A05_BCS-CB38799_25.0           7         140          25   

                                 AOB78_gp04  AOB78_gp05  AOB78_gp06  \
Metadata_sample_id                                                    
plate1_24h_A01_DMSO_25.0                  0          57           9   
plate1_24h_A02_DMSO_25.0                  1          66          17   
plate1_24h_A03_BCS-DH19921_25.0           0          39           8   
plate1_24h_A04_BCS-DH19921_2.5            2          56          14   
plate1_24h_A05_BCS-CB38799_25.0           2          82           8   

    

In [12]:
import os
os.system(f"cp {start_genestack_table} {experiment_title}.xlsx")

0

In [13]:
meta_df = pd.read_excel(metadata, sheet_name="Barcodes", index_col=0)
def create_long_format(tmp_df, value_name):
    tmp_df["index"] = tmp_df.index
    long_df = tmp_df.reset_index().melt(id_vars='index', var_name='Column', value_name=value_name)
    long_df = long_df[(long_df["Column"] != "Barcodes") & (long_df["Column"] != "level_0")] 
    long_df['Metadata_Wells'] = long_df['index'] + long_df['Column'].astype(str).str.zfill(2)
    return long_df.drop(columns=['index', 'Column'])
meta_df = create_long_format(meta_df, "Metadata_Barcodes")

In [14]:
df = df.merge(meta_df, on="Metadata_Wells", how="left")

In [15]:
plate_info = pd.read_excel(sit_table_alithea, sheet_name = "Sheet1")
plate_dict = dict(zip(plate_info["Plate.ID"], plate_info["LibraryName"]))

In [16]:
df["LibraryName"] = df["Metadata_plate"].map(plate_dict)
if "Metadata_Barcodes_x" in df.columns:
    df.rename(columns={"Metadata_Barcodes_x": "Metadata_Barcodes"}, inplace=True)

In [17]:
link_p360_ids = "https://genestack-odm.velocity.ag/ui/study/GSF000471"

In [18]:
import pandas as pd

# Step 1: Load the Excel file
file_path = f'{experiment_title}.xlsx'  # Replace with your file path
sheet_name = 'SAMPLE'  # Replace with the name of the sheet you want to modify

# Step 2: Read the specific sheet
gs_df = pd.read_excel(file_path, sheet_name=sheet_name, header=0, skiprows=3)

In [21]:
new_table = []
for index, row in df.iterrows():
    new_row = gs_df.iloc[0].copy()
    new_row['Sample Source ID'] = row["Metadata_sample_id"]
    new_row['Treatment / BSC code'] = row["Metadata_treatments"]
    new_row['Treatment / Dosage unit'] = "Mikromolar"
    new_row['Treatment / Dosage'] = row["Metadata_concentration"]
    new_row['Sampling time point / Unit'] = "Hours"
    new_row['Sampling time point / Timepoint'] = row["Metadata_timepoint"]
    new_row['Design / Well position X'] = row["Metadata_Wells"][0]
    new_row['Design / Well position Y'] = int(row["Metadata_Wells"][1:])
    new_row['Alithea / Barcode'] = row["Metadata_Barcodes"]
    new_row['Alithea / LibraryName'] = row["LibraryName"]
    new_table.append(new_row)
    
new_table_df = pd.DataFrame(new_table).reset_index(drop=True)
new_table_df.columns = new_row.index

In [ ]:
#df.to_csv("full_raw_matrix_and_meta.csv", index=False)

In [23]:
# Step 4: Save the modified DataFrame along with all original sheets
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    # Write the modified DataFrame to the same sheet
    new_table_df.to_excel(writer, sheet_name=sheet_name, index=False)